***В этом примере используются дополнительные модули для вывода моделей и визуализации процесса обучения. Установите их, если они отсутствуют в вашей среде***

In [ ]:
!pip install numpy
!pip install tqdm
!pip install matplotlib 
!pip install pytorch_msssim

## Пример использования TorchCNNBuilder для датасета MovingMnist

#### Набор данных MovingMnist является классическим для задачи прогнозирования видео. Он представлен 1000 выборками с 20 сериями кадров с числами, движущимися по разным траекториям. Его можно загрузить по [официальной ссылке](https://www.cs.toronto.edu/~nitish/unsupervised_video/) или используя код следующей ячейки.

In [ ]:
import os
import urllib.request
import numpy as np

# Создаём директорию для данных, если её нет
os.makedirs('data', exist_ok=True)

# Загружаем датасет Moving MNIST
url = 'https://www.cs.toronto.edu/~nitish/unsupervised_video/mnist_test_seq.npy'
filename = 'data/moving_mnist.npy'

if not os.path.exists(filename):
    print("Загрузка датасета Moving MNIST...")
    urllib.request.urlretrieve(url, filename)
    print("Загрузка завершена!")
else:
    print("Датасет уже существует, загрузка не требуется.")

# Загружаем данные для проверки
try:
    data = np.load(filename)
    print(f"Датасет успешно загружен! Размерность: {data.shape}")
except Exception as e:
    print(f"Ошибка при загрузке датасета: {e}")

In [1]:
import numpy as np
import torch
from matplotlib import pyplot as plt
from torch import nn, optim, tensor
from torch.utils.data import TensorDataset, DataLoader
from torchcnnbuilder.models import ForecasterBase

##### Подготовка данных включает нормализацию и разделение на обучающую и тестовую части. В качестве признаков (входных) для модели используются первые 17 кадров, в качестве целевых (выходных) используются последние 3 кадра.

Следует отметить, что в каждом временно-пространственном ряду есть **нециклическая компонента**. Поэтому модель должна изучать динамику перемещения чисел на примерах из других рядов (на других числах). А сверточные слои должны помочь воспроизвести вид числа по предыдущим кадрам ряда.

In [2]:
data = np.load('data/moving_mnist.npy').astype(np.float32)/255 # в силу размера данных они не расположены в репозитории, их можно скачать по ссылке выше

train_set = data[:, :8000, :, :]
test_set = data[:, 8000:, :, :]

train_features = train_set[:17, :, :, :]
train_features = np.swapaxes(train_features, 0, 1)
train_target = train_set[17:, :, :, :]
train_target = np.swapaxes(train_target, 0, 1)

train_dataset = TensorDataset(tensor(train_features), tensor(train_target))

##### Построение модели с простой структурой - 5 сверточных и 5 транспонированных сверточных слоев. Разрешение изображений 64x64 пикселя

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Рассчет на устройстве: {device}')
model = ForecasterBase(input_size=[64, 64],
                       in_time_points=17,
                       out_time_points=3,
                       n_layers=5)
model = model.to(device)

##### Установка параметров для обучения. Представлена простая стратегия без планировщика. Количество эпох и размер батча могут быть изменены в зависимости от устройства и требований к качеству.

In [4]:
epochs = 2000
batch_size = 100
dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
optimizer = optim.AdamW(model.parameters(), lr=0.0001)
criterion = nn.L1Loss()

##### Обучение модели на 8000 образцах. Значения функции потерь на каждой эпохе сохраняется для визуализации сходимости


In [5]:
from tqdm.notebook import tqdm

progress_bar = tqdm(list(np.arange(epochs)), desc="Epoch", colour="white")
info_bar = {"Loss": 0}
losses = []
epoches = []
for epoch in range(epochs):
    loss = 0
    for train_features, train_targets in dataloader:
        train_features = train_features.to(device)
        train_targets = train_targets.to(device)

        optimizer.zero_grad()
        outputs = model(train_features)

        train_loss = criterion(outputs, train_targets)

        train_loss.backward()
        optimizer.step()

        loss += train_loss.item()

    loss = loss / len(dataloader)
    
    info_bar['Loss'] = np.round(loss, 5)
    progress_bar.update()
    progress_bar.set_postfix_str(info_bar)

    losses.append(loss)
    epoches.append(epoch)

torch.save(model.state_dict(), f'mnist_{epochs}.pt')

##### Визуализация значения функции потерь по эпохам. Постепенное уменьшение значения функции потерь свидетельствует о том, что задача поставлена правильно.

In [6]:
plt.plot(epoches, losses)
plt.grid()
plt.xlabel('Эпоха')
plt.ylabel('L1 Loss')
plt.title('График сходимости')
plt.show()

### Оценка качества на тестовой выборке
##### *Загрузка предикторов и целевых полей для тестовой выборки* 

In [9]:
test_features = test_set[:17, :, :, :]
test_features = np.swapaxes(test_features, 0, 1)
test_target = test_set[17:, :, :, :]
test_target = np.swapaxes(test_target, 0, 1)
print('Данные загружены')

##### *MAE (средняя абсолютная ошибка) - расчет для каждого элемента тестовой выборки*

In [23]:
l1_errors = []
for s in range(test_features.shape[0]):
    features = tensor(test_features[s]).to(device)
    prediction = model(features).detach().cpu().numpy()
    target = test_target[s]
    mae = np.mean(abs(prediction - target))
    l1_errors.append(mae)
print(f'Mean MAE for test set = {np.mean(l1_errors)}')  

##### Визуализация результата предсказания для тестовой выборки (для первых пяти образцов) 

In [24]:
for s in range(5):
    tensor_features = tensor(test_features[s]).to(device)
    prediction = model(tensor_features).detach().cpu().numpy()
    plt.rcParams["figure.figsize"] = (8, 6)
    fig, axs = plt.subplots(2, 3)
    for i in range(3):
        axs[0][i].imshow(prediction[i])
        axs[1][i].imshow(test_target[s][i])
    plt.suptitle(f'Образец {s}')
    plt.show()

На основе визуализации можно сделать вывод, что предсказательная способность такой модели с описанной схемой обучения ограничена 2 кадрами, несмотря на высокую метрику качества.